### Overview

Running the ALM helper notebook will automate reconfiguring deployment parameters, updating notebook metadata, creating delta tables, creating folder structure identical to the bronze Lakehouse folders*, and copying sample data. This notebook does not facilitate copying or moving any customer data. It does not replicate folder structures found in the Silver or Gold Lakehouses.

<mark>**NOTE**: If using the Fabric UI, make sure to attach the source admin lakehouse before starting a session.</mark>

Start by providing information about the source and destination workspaces in the Parameters section below and then execute the remaining cells in the notebook.

### Parameters

In the cell below, please provide the following parameters to start the migration process. These will be the only inputs required for the migration.

___source_workspace_id___: The id of the source workspace.<br>
___source_admin_lakehouse_id___: The lakehouse id for the administration lakehouse in the source workspace.<br>
___dest_workspace_id___: The id of the destination workspace.<br>
___dest_admin_lakehouse_id___: The lakehouse id for the administration lakehouse in the destination workspace.<br>
___solution_name___: The name of the HDS solution in the source workspace, for example "healthcare1".

In [ ]:
source_workspace_id = ""
source_admin_lakehouse_id = ""
dest_workspace_id = ""
dest_admin_lakehouse_id = ""
solution_name=""

#### Helper Methods

Run the following cell to register required helper methods.

In [ ]:
from typing import Dict, Any
from sempy.fabric import FabricRestClient
from delta import DeltaTable
import json
import time
import base64
import nbformat

def get_bronze_lakehouse_paths(source_workspace_id, source_admin_lakehouse_id):
    """Summary: Get a flat list of the directories in a lakehouse

    Args:
        source_workspace_id (str): The source workspace id
        source_admin_lakehouse_id (str): The source admin lakehouse id
    """
    deployment_parameters_config = get_deployment_parameters_config(source_workspace_id, source_admin_lakehouse_id)
    bronze_lakehouse_data = get_lakehouse(source_workspace_id, deployment_parameters_config['activitiesGlobalParameters']['bronze_lakehouse_id'])
    
    lakehouse_dfs = get_lakehouse_dfs_domain(bronze_lakehouse_data)
    bronze_lakehouse_id = bronze_lakehouse_data['id']
    source_files_path = f"abfss://{source_workspace_id}@{lakehouse_dfs}/{bronze_lakehouse_id}/Files/"

    file_infos = mssparkutils.fs.ls(source_files_path)
    paths = []
    while len(file_infos) > 0:

        current_file_info = file_infos.pop(0)
        
        # Add path to list
        if current_file_info.isDir:
            paths.append("Files/" + current_file_info.path.split("Files/")[-1])
        
        # Add children to queue
        for f in mssparkutils.fs.ls(current_file_info.path):
            if f.isDir:
                file_infos.append(f)

    return paths

def create_bronze_folders(source_workspace_id: str, source_admin_lakehouse_id: str, dest_workspace_id: str, solution_name: str = "healthcare1") -> None:
    """Summary: Create initial folder structure in the destination Bronze Lakehouse

    Args:
        source_workspace_id (str): The source workspace id
        source_admin_lakehouse_id (str): The source admin lakehouse id
        dest_workspace_id (str): The destination workspace id
        solution_name (str, optional): The name of the HDS solution. Defaults to "healthcare1".
    """
    dest_lakehouses = get_workspace_lakehouses(dest_workspace_id)
    target_lakehouse = None
    for lh_name in dest_lakehouses.keys():
        if lh_name == f"{solution_name}_msft_bronze":
            target_lakehouse = dest_lakehouses[lh_name]
            break

    if target_lakehouse is None:
        print("Bronze lakehouse not found, not creating folders")
    else:
        print(json.dumps(target_lakehouse))
        target_lakehouse_id = target_lakehouse["id"]
        target_lakehouse_data = get_lakehouse(dest_workspace_id, target_lakehouse_id)
        lakehouse_dfs = get_lakehouse_dfs_domain(target_lakehouse_data)
        source_file_path = f"abfss://{dest_workspace_id}@{lakehouse_dfs}/{target_lakehouse_id}/"
        for sub_path in get_bronze_lakehouse_paths(source_workspace_id, source_admin_lakehouse_id):
            print(source_file_path + sub_path)
            mssparkutils.fs.mkdirs(source_file_path + sub_path)

def get_deployment_parameters_config(source_workspace_id: str, source_admin_lakehouse_id: str) -> Any:
    """Summary: A helper method to get the deployment parameters configuration.

    Args:
        source_workspace_id (str): The source workspace id.
        source_admin_lakehouse_id (str): The source admin lakehouse id.

    Returns:
        Any: The deployment parameters config as a json object.
    """
    deployment_parameters_configuration_source_path = "Files/system-configurations/deploymentParametersConfiguration.json"

    source_lakehouse_data = get_lakehouse(source_workspace_id, source_admin_lakehouse_id)
    source_dfs_domain = get_lakehouse_dfs_domain(source_lakehouse_data)
    configuration_path = f"abfss://{source_workspace_id}@{source_dfs_domain}/{source_admin_lakehouse_id}/{deployment_parameters_configuration_source_path}"
    df = spark.read.option("multiline", "true").json(configuration_path)
    deployment_parameters_config_json = df.collect()[0].asDict(recursive=True)
    return deployment_parameters_config_json

def get_workspace_artifacts_by_id(workspace_id: str) -> Dict:
    """Summary: A helper method to get a dictionary of artifacts in a specified workspace.

    Args:
        workspace_id (str): The workspace id.

    Returns:
        Dict: A dictionary mapping workspace artifacts from their id to additional metadata
    """
    fabric_client = FabricRestClient()
    artifacts = fabric_client.get(f"v1/workspaces/{workspace_id}/items").json()['value']
    source_workspace_artifacts = { artifact["id"]: artifact for artifact in artifacts }
    return source_workspace_artifacts

def get_workspace_lakehouses(workspace_id: str) -> Dict:
    """Summary: A helper method to get a dictionary of Lakehouse artifacts in a specified workspace.

    Args:
        workspace_id (str): The workspace id.

    Returns:
        Dict: A dictionary of lakehouses in a specified workspace.
    """
    fabric_client = FabricRestClient()
    lakehouses = fabric_client.get(f"v1/workspaces/{workspace_id}/items?type=Lakehouse").json()['value']
    lakehouses_dict = { lakehouse["displayName"]: lakehouse for lakehouse in lakehouses }
    return lakehouses_dict

def get_workspace_notebooks(workspace_id: str) -> Dict:
    """Summary: A helper method to get a dictionary of Notebook artifacts in a specified workspace.

    Args:
        workspace_id (str): The workspace id.

    Returns:
        Dict: A dictionary of notebooks in a specified workspace.
    """
    fabric_client = FabricRestClient()
    notebooks = fabric_client.get(f"v1/workspaces/{workspace_id}/items?type=Notebook").json()['value']
    notebooks_dict = { nb["displayName"]: nb for nb in notebooks }
    return notebooks_dict

def get_workspace_environments(workspace_id: str) -> Dict:
    """Summary: A helper method to get a dictionary of Environment artifacts in a specified workspace.

    Args:
        workspace_id (str): The workspace id.

    Returns:
        Dict: A dictionary of environments in a specified notebooks.
    """
    fabric_client = FabricRestClient()
    environments = fabric_client.get(f"v1/workspaces/{workspace_id}/items?type=Environment").json()['value']
    environments_dict = { env["displayName"]: env for env in environments }
    return environments_dict

def get_workspace_healthcare_data_solution_by_name(workspace_id: str, solution_name: str|None = "healthcare1") -> Any | None:
    """Summary: A helper method to get a healthcare data solution artifact in a workspace by name.

    Args:
        workspace_id (str): The workspace id.
        solution_name (str | None, optional): The name of the solution. Defaults to "healthcare1".

    Returns:
        Any: The healthcare data solution artifact in found, None otherwise.
    """
    fabric_client = FabricRestClient()
    artifacts = fabric_client.get(f"v1/workspaces/{workspace_id}/items?type=Healthcaredatasolution").json()['value']
    for artifact in artifacts:
        if artifact["displayName"] == solution_name:
            return artifact
    return None

def get_workspace_details(workspace_id: str) -> Any:
    """Summary: A helper method to get the workspace details.

    Args:
        workspace_id (str): The workspace id.

    Returns:
        Any: Workspace details.
    """
    fabric_client = FabricRestClient()
    workspace_details = fabric_client.get(f"/v1/workspaces/{workspace_id}").json()
    return workspace_details

def get_updated_global_activity_parameters(
        source_workspace_id: str,
        source_admin_lakehouse_id:str,
        dest_workspace_id: str,
        dest_admin_lakehouse_id: str,
        solution_name="healthcare1") -> Any:
    
    """Summary: A helper method to update the global activity parameters for the deployment parameters config in a destination workspace.

    Args:
        source_workspace_id (str): The source workspace id.
        source_admin_lakehouse_id (str): The admin lakehouse.
        dest_workspace_id (str): The destination workspace id.
        dest_admin_lakehouse_id (str): The destination admin lakehouse id.
        solution_name (str, optional): The healthcare data solution name. Defaults to "healthcare1".

    Returns:
        Any: The updated global activity parameters with resolved values.
    """
    
    deployment_parameters_config = get_deployment_parameters_config(source_workspace_id, source_admin_lakehouse_id)
    global_parameters = deployment_parameters_config["activitiesGlobalParameters"]

    source_workspace_artifacts = get_workspace_artifacts_by_id(source_workspace_id)
    dest_workspace_artifacts = get_workspace_artifacts_by_id(dest_workspace_id)

    source_workspace_lakehouses = { v["displayName"]: v for v in source_workspace_artifacts.values() if str(v['type']).lower() == "lakehouse" }
    dest_workspace_lakehouses = { v["displayName"]: v for v in dest_workspace_artifacts.values() if str(v['type']).lower() == "lakehouse" }
    dest_workspace_artifacts_by_name = { v["displayName"]: v for v in dest_workspace_artifacts.values() }

    updated_global_parameters = {}
    replacements = []
    
    # Iterate over global parameters in the source config
    for pk, pv in global_parameters.items():

        # Keep track of the update global parameters for the destination config
        updated_global_parameters[pk] = pv

        if source_workspace_id in updated_global_parameters[pk]:
            updated_global_parameters[pk] = str(updated_global_parameters[pk]).replace(source_workspace_id, dest_workspace_id)

        # Update lakehouse ids with those found in the destination lakehouse
        if "_lakehouse_id" in pk:
            lakehouse_name = pk.split("_lakehouse_id")[0]
            target_notebook = f"{solution_name}_msft_{lakehouse_name.lower()}"
            if target_notebook in source_workspace_lakehouses and target_notebook in dest_workspace_lakehouses:
                updated_global_parameters[pk] = dest_workspace_lakehouses[target_notebook]["id"]

        # Try to swap ids when the id present in the artifact name exists in both
        # the source workspace and destination workspace.
        for id in source_workspace_artifacts.keys():
            if id in pv:
                source_artifact_name = source_workspace_artifacts[id]["displayName"]

                if source_artifact_name in dest_workspace_artifacts_by_name:
                    replacement_id = dest_workspace_artifacts_by_name[source_artifact_name]["id"]
                    replacements.append([source_artifact_name ,replacement_id])
                    updated_global_parameters[pk] = str(updated_global_parameters[pk]).replace(id, replacement_id)

    return updated_global_parameters, replacements

def get_updated_activties_configuration(source_workspace_id: str, source_admin_lakehouse_id: str, dest_workspace_id: str, dest_admin_lakehouse_id: str) -> Any:
    """Summary: A helper method to update the activities config for the deployment parameters config in a destination workspace.

    Args:
        source_workspace_id (str): The source workspace id.
        source_admin_lakehouse_id (str): The admin lakehouse.
        dest_workspace_id (str): The destination workspace id.
        dest_admin_lakehouse_id (str): The destination admin lakehouse id.

    Returns:
        Any: The updated activities with resolved values.
    """
    
    deployment_parameters_config = get_deployment_parameters_config(source_workspace_id, source_admin_lakehouse_id)
    activities_json = deployment_parameters_config['activities']

    source_workspace_artifacts = get_workspace_artifacts_by_id(source_workspace_id)
    dest_workspace_artifacts = get_workspace_artifacts_by_id(dest_workspace_id)

    dest_workspace_notebooks = {v["displayName"]: v["id"] for v in dest_workspace_artifacts.values() if str(v['type']).lower() == "notebook" }
    dest_workspace_artifacts_by_name = {v["displayName"]: v for v in dest_workspace_artifacts.values() }

    configuration_activities = {}
    for activity in activities_json.keys():
        configuration_activities[activities_json[activity]["name"]] = activities_json[activity]

    updated_activities = {}
    replacements = []
    
    # Iteratate over the activies found in the source config
    for config_activity_name, config_activity in configuration_activities.items():

        # If the activity (notebook) name is found in the destination workspace
        if config_activity_name in dest_workspace_notebooks:

            new_activity_id = dest_workspace_notebooks[config_activity_name]
            config_parameters = config_activity["parameters"]
            updated_parameters = {}

            # For each parameter in the activty parameters section
            for pk, pv in config_parameters.items():
                updated_parameters[pk] = pv

                # Replace the source workspace id with the destination workspace id if found
                if source_workspace_id in pv:
                    updated_parameters[pk] = str(updated_parameters[pk]).replace(source_workspace_id, dest_workspace_id)

                # Try to find ids in the parameters
                for id in source_workspace_artifacts.keys():
                    if id in pv:
                        source_artifact_name = source_workspace_artifacts[id]["displayName"]
                        if source_artifact_name in dest_workspace_artifacts_by_name:
                            replacement_id = dest_workspace_artifacts_by_name[source_artifact_name]["id"]
                            replacements.append([source_artifact_name ,replacement_id])
                            updated_parameters[pk] = str(updated_parameters[pk]).replace(id, replacement_id)
            
            updated_activities[new_activity_id] = {
                "name": config_activity_name,
                "parameters": updated_parameters
            }

    return updated_activities, replacements

def consolidate_replacements(global_param_replacements: Any, activity_replacements: Any):
    """Summary: A helper method to consolidate all of the replaces that were made in updating the deployment parameters config.

    Args:
        global_param_replacements (Any): Replacements made in the global parameters section.
        activity_replacements (Any): Replacements made in the activity section.

    Returns:
        Any: A consolidated dict of all replacements. 
    """
    all_replacements = {}
    for r in activity_replacements:
        if r[0] not in all_replacements:
            all_replacements[r[0]] = r[1]
    
    for r in global_param_replacements:
        if r[0] not in all_replacements:
            all_replacements[r[0]] = r[1]
    
    return all_replacements

def get_updated_deployment_parameters_configuration(
    source_workspace_id: str,
    source_admin_lakehouse_id: str,
    dest_workspace_id: str,
    dest_admin_lakehouse_id: str,
    solution_name="healthcare1") -> Any:
    """Summary: A helper method to update the deployment parameters configuration.

    Args:
        source_workspace_id (str): The source workspace id.
        source_admin_lakehouse_id (str): The admin lakehouse.
        dest_workspace_id (str): The destination workspace id.
        dest_admin_lakehouse_id (str): The destination admin lakehouse id.
        solution_name (str, optional): The healthcare data solution name. Defaults to "healthcare1".

    Returns:
        Any: The updated deployment parameters config.
    """
    
    updated_gloabal_parameters, global_param_replacements = get_updated_global_activity_parameters(source_workspace_id, source_admin_lakehouse_id, dest_workspace_id, dest_admin_lakehouse_id, solution_name)
    updated_activities, activity_replacements = get_updated_activties_configuration(source_workspace_id, source_admin_lakehouse_id, dest_workspace_id, dest_admin_lakehouse_id)

    updated_deployment_parameters_configuration = {
        "activitiesGlobalParameters": updated_gloabal_parameters,
        "activities": updated_activities
    }
    
    return updated_deployment_parameters_configuration, consolidate_replacements(global_param_replacements, activity_replacements)

def get_system_configurations_path(workspace_id: str, admin_lakehouse_id: str) -> str:
    lakehouse_data = get_lakehouse(workspace_id, admin_lakehouse_id)
    lakehouse_dfs_domain = get_lakehouse_dfs_domain(lakehouse_data)

    return f"abfss://{workspace_id}@{lakehouse_dfs_domain}/{admin_lakehouse_id}/Files/system-configurations"

def get_deployment_parameters_destination_path(dest_workspace_id: str, dest_admin_lakehouse_id: str) -> str:
    return get_system_configurations_path(dest_workspace_id, dest_admin_lakehouse_id) + "/deploymentParametersConfiguration.json"

def copy_system_configuration_files(dest_workspace_id: str, dest_admin_lakehouse_id: str) -> str:
    destination_lakehouse_data = get_lakehouse(dest_workspace_id, dest_admin_lakehouse_id)
    destination_lakehouse_dfs_domain = get_lakehouse_dfs_domain(destination_lakehouse_data)

    return f"abfss://{dest_workspace_id}@{destination_lakehouse_dfs_domain}/{dest_admin_lakehouse_id}/Files/system-configurations/deploymentParametersConfiguration.json"

def get_lakehouse_dfs_domain(lakehouse_data: Any) -> str:
    """Summary: A helper method for extracting the dfs domain from lakehouse metdata.

    Args:
        lakehouse_data (Any): The name of the lakehouse.

    Returns:
        str: The dfs domain
    """
    onelake_files_path = lakehouse_data["properties"]["oneLakeFilesPath"]
    env = onelake_files_path.split("//")[1].split("/")[0]
    return env

def get_lakehouse(workspace_id: str, lakehouse_id: str) -> Any:
    """Summary: A helper method to get a lakehouse using the workspace id and lakehouse id.

    Args:
        workspace_id (str): The workspace id.
        lakehouse_id (str): The lakehouse id.

    Returns:
        Any: Lakehouse metadata.
    """
    fabric_client = FabricRestClient()
    return fabric_client.get(f"/v1/workspaces/{workspace_id}/lakehouses/{lakehouse_id}").json()

def copy_system_data(
    source_workspace_id: str,
    source_admin_lakehouse_id: str,
    dest_workspace_id: str,
    dest_admin_lakehouse_id: str,
    solution_name: str) -> None:
    """Summary: A helper method for copying important system metadata to the destination workspace.

    Args:
        source_workspace_id (str): The source workspace id.
        source_admin_lakehouse_id (str): The admin lakehouse.
        dest_workspace_id (str): The destination workspace id.
        dest_admin_lakehouse_id (str): The destination admin lakehouse id.
        solution_name (str, optional): The healthcare data solution name. Defaults to "healthcare1".
    """
    try:
        internal_relative_path = "/system-configurations/HDS/_internal"
        libraries_relative_path = "/deployment-assets/libraries"

        dest_admin_lakehouse_data = get_lakehouse(dest_workspace_id, dest_admin_lakehouse_id)

        lakehouse_dfs = get_lakehouse_dfs_domain(dest_admin_lakehouse_data)

        dest_admin_lakehouse_id = dest_admin_lakehouse_data["id"]

        source_file_path = f"abfss://{source_workspace_id}@{lakehouse_dfs}/{source_admin_lakehouse_id}/Files"
        dest_files_path = f"abfss://{dest_workspace_id}@{lakehouse_dfs}/{dest_admin_lakehouse_id}/Files"

        if notebookutils.fs.exists(f"{source_file_path}{internal_relative_path}") and notebookutils.fs.exists(dest_files_path):
            print("Copying internal resources...")
            notebookutils.fs.fastcp(f"{source_file_path}{internal_relative_path}", f"{dest_files_path}{internal_relative_path}", recurse=True)
            print("Successfully copied internal resources to destination workspace.")
        else:
            print("Internal data not found in administrative lakehouse or destination does not exist")
        
        if notebookutils.fs.exists(f"{source_file_path}{libraries_relative_path}") and notebookutils.fs.exists(dest_files_path):
            print("Copying libraries...")
            notebookutils.fs.fastcp(f"{source_file_path}{internal_relative_path}", f"{dest_files_path}{internal_relative_path}", recurse=True)
            print("Successfully copied libraries to destination workspace.")
        else:
            print("HDS Libraries not found in administrative lakehouse or destination does not exist")
    except Exception as ex:
        print("Exception occurred when copying system data: {ex}")

def copy_data_assets(source_workspace_id: str, dest_workspace_id: str, solution_name: str) -> None:
    """Summary: A helper method for copying important data assets to the destination workspace.

    Args:
        source_workspace_id (str): The source workspace id.
        dest_workspace_id (str): The destination workspace id.
        solution_name (str): The name of the healthcare data solution.
    """

    sample_data_relative_path = "/SampleData"
    reference_data_relative_path = "/ReferenceData"
    source_workspace_lakehouses = get_workspace_lakehouses(source_workspace_id)
    dest_workspace_lakehouses = get_workspace_lakehouses(dest_workspace_id)
    bronze_lakehouse_name = f"{solution_name}_msft_bronze"

    if bronze_lakehouse_name in source_workspace_lakehouses and bronze_lakehouse_name in dest_workspace_lakehouses:

        source_bronze_lakehouse_id = source_workspace_lakehouses[bronze_lakehouse_name]["id"]
        dest_bronze_lakehouse_id = dest_workspace_lakehouses[bronze_lakehouse_name]["id"]
        source_lakehouse = get_lakehouse(source_workspace_id, source_bronze_lakehouse_id)
        dest_lakehouse = get_lakehouse(dest_workspace_id, dest_bronze_lakehouse_id)

        source_lakehouse_dfs = get_lakehouse_dfs_domain(source_lakehouse)
        dest_lakehouse_dfs = get_lakehouse_dfs_domain(dest_lakehouse)

        source_file_path = f"abfss://{source_workspace_id}@{source_lakehouse_dfs}/{source_bronze_lakehouse_id}/Files"
        dest_files_path = f"abfss://{dest_workspace_id}@{dest_lakehouse_dfs}/{dest_bronze_lakehouse_id}/Files"

        print(dest_files_path)

        try:
            if notebookutils.fs.exists(f"{source_file_path}{sample_data_relative_path}") and notebookutils.fs.exists(dest_files_path):
                print("Copying sample data...")
                notebookutils.fs.fastcp(f"{source_file_path}{sample_data_relative_path}", f"{dest_files_path}", recurse=True)
                print("Successfully copied sample data to destination workspace.")
            else:
                print("Sample data not found in {bronze_lakehouse_name} or destination does not exist")
        except Exception as ex:
            print(f"Exception occurred while copying sample data, the source or destination folder might not exist.")

        try:
            if notebookutils.fs.exists(f"{source_file_path}{reference_data_relative_path}") and notebookutils.fs.exists(dest_files_path):
                print("Copying reference data...")
                notebookutils.fs.fastcp(f"{source_file_path}{reference_data_relative_path}", f"{dest_files_path}", recurse=True)
                print("Successfully copied reference data to destination workspace.")
            else:
                print("Reference data not found in {bronze_lakehouse_name} or destination does not exist")
        except Exception as ex:
            print(f"Exception occurred while copying reference data, the source or destination folder might not exist.")

    else:
        print(f"{bronze_lakehouse_name} not found in either the source or target workspace. Sample data and reference data were not copied.")

def copy_workload_system_data(source_workspace_id: str, dest_workspace_id: str, dest_admin_lakehouse_id: str, solution_name: str) -> None:

    """Summary: A helper method for copying important workload data to the destination workspace.

    Args:
        source_workspace_id (str): The source workspace id.
        dest_workspace_id (str): The destination workspace id.
        solution_name (str): The name of the healthcare data solution.
    """

    try:
        internal_relative_path = "/DMHConfiguration"
        libraries_relative_path = "/deployment-assets/libraries"

        dest_admin_lakehouse_data = get_lakehouse(dest_workspace_id, dest_admin_lakehouse_id)
        lakehouse_dfs = get_lakehouse_dfs_domain(dest_admin_lakehouse_data)

        source_solution_id = get_workspace_healthcare_data_solution_by_name(source_workspace_id, solution_name)["id"]
        dest_solution_id = get_workspace_healthcare_data_solution_by_name(dest_workspace_id, solution_name)["id"]

        source_files_path = f"abfss://{source_workspace_id}@{lakehouse_dfs}/{source_solution_id}"
        dest_files_path = f"abfss://{dest_workspace_id}@{lakehouse_dfs}/{dest_solution_id}"

        print(source_files_path)
        print(dest_files_path)

        try:
            if notebookutils.fs.exists(f"{source_files_path}{internal_relative_path}"):
                print("Copying internal resources...")
                notebookutils.fs.fastcp(f"{source_files_path}{internal_relative_path}", f"{dest_files_path}", recurse=True)
                print("Successfully copied internal resources to destination workspace.")
            else:
                print("Internal data not found in workload or does not exist")
        except Exception as ex:
            print(f"Exception occurred while copying internal sources, the source or destination folder might not exist.")
        
        try:
            if notebookutils.fs.exists(f"{source_files_path}{libraries_relative_path}"):
                print("Copying libraries...")
                notebookutils.fs.fastcp(f"{source_files_path}{libraries_relative_path}", f"{dest_files_path}/deployment-assets", recurse=True)
                print("Successfully copied libraries to destination workspace.")
            else:
                print("HDS Libraries not found in administrative lakehouse or destination does not exist")
        except Exception as ex:
            print(f"Exception occurred while copying libraries, the source or destination folder might not exist.")

    except Exception as ex:
        print(f"Exception occurred when copying workload system data: {ex}")

def get_target_environment(workspace_id: str, solution_name: str) -> Any | None:
    """Summary: Get the environment associated with a healthcare data solution

    Args:
        workspace_id (str): _description_
        solution_name (str): _description_

    Returns:
        Any: The environment metadata if found, None otherwise
    """
    environments = get_workspace_environments(workspace_id)

    for env in environments.keys():
        if solution_name in env:
            return environments[env]
    return None

def get_notebook_definition(dest_workspace_id: str, dest_notebook_id: str) -> Any:
    """Summary: A helper method to get the notebook definition

    Args:
        dest_workspace_id (str): The destination workspace id.
        dest_notebook_id (str): The notebook id.

    Returns:
        Any: The notebook definition
    """

    fabric_client = FabricRestClient()
    response = fabric_client.post(f"/v1/workspaces/{dest_workspace_id}/items/{dest_notebook_id}/getDefinition?format=ipynb")

    # Post call will likely start a long running operation to download the notebook content
    if response.status_code == 200:
        print("Downloaded notebook definition")
        return response.json()
    elif response.status_code == 202:
        print("Started notebook download operation")
    else:
        print("Get notebook definition failed, exiting")
        return None
    
    # Get the download oepration route and poll until complete
    if "Location" in response.headers:
        lro_route = "v1" + response.headers["Location"].split("/v1")[1]

        lro_complete = False
        status = "not_started"
        while lro_complete == False:
            lro_response = fabric_client.get(lro_route)
            lro_json = lro_response.json()
            if 'status' in lro_json:
                status = lro_json['status'].lower()
                print(f"Download notebook status: {status}")
                if status == 'succeeded':
                    lro_complete = True
                elif status == 'failed':
                    lro_complete = True
                else:
                    print("Download operation in progress, polling again...")
                    time.sleep(1)
            else:
                print("Status not found in download operation response")
                lro_complete = True
        
        # Download the notebook definition using the result endpoint
        if lro_complete and status == "succeeded":
            lro_result_response = fabric_client.get(lro_route + "/result")
            lro_json = lro_result_response.json()
            return lro_json
    
    return {}

def get_notebook_json_from_definition(notebook_definition: Any) -> Any:
    """Summary: Get the notebook json content from a notebook definition.

    Args:
        notebook_definition (Any): The notebook definition object.

    Returns:
        Any: The notebook content as json
    """
    definition_payload = notebook_definition['definition']['parts'][0]['payload']
    decoded_payload = base64.b64decode(definition_payload)
    nb_json = json.loads(decoded_payload)
    return nb_json

def upload_notebook(workspace_id: str, notebook_id: str, notebook_name: str, initial_definition: Any, updated_notebook: Any) -> None:
    """Summary: A helper method to upload an updated notebook.

    Args:
        workspace_id (str): The workspace id.
        notebook_id (str): The notebook id.
        notebook_name (str): The notebook name.
        initial_definition (Any): The initial (source) notebook definition.
        updated_notebook (Any): The updated notebook content.
    """
        
    fabric_client = FabricRestClient()
    encoded_payload = base64.b64encode(json.dumps(updated_notebook).encode('utf-8')).decode('utf-8')

    initial_definition["definition"]["format"] = "ipynb"
    initial_definition["definition"]["parts"] = [
        {
            "path": notebook_name + ".py",
            "payload": encoded_payload,
            "payloadType": "InlineBase64"
        }
    ]

    url = f"/v1/workspaces/{workspace_id}/notebooks/{notebook_id}/updateDefinition"
    response = fabric_client.post(url, json=initial_definition)
    if response.ok:
        print(f"Notebook {notebook_name} updated started")
        
    else:
        print("Error updating notebook")
        print(response.status_code)
        print(response.content)

def update_notebooks(source_workspace_id: str, dest_workspace_id: str, solution_name = "healthcare1") -> None:
    """Summary: A helper method to updated notebooks in the destination workspace.

    Args:
        source_workspace_id (str): The source workspace id.
        dest_workspace_id (str): The destination workspace id.
        solution_name (str, optional): The healthcare data solution name. Defaults to "healthcare1".
    """
    source_notebooks_dict = get_workspace_notebooks(source_workspace_id)
    dest_notebooks_dict = get_workspace_notebooks(dest_workspace_id)
    dest_lakehouses = get_workspace_lakehouses(dest_workspace_id)
    dest_environment = get_target_environment(dest_workspace_id, solution_name)
    dest_solution = get_workspace_healthcare_data_solution_by_name(dest_workspace_id, solution_name)

    env_to_attach = {
        "environmentId": dest_environment["id"] if dest_environment else None,
        "workspaceId": dest_workspace_id,
    }

    print("Destination Lakehouses:")
    print(json.dumps(dest_lakehouses, indent=2))

    fabric_client = FabricRestClient()

    for notebook_name in dest_notebooks_dict.keys():
        
        # Include notebooks that are prefixed with the solution
        if solution_name in notebook_name and "migrate" not in notebook_name:
            print(f"Attempting to update {notebook_name}...")

            dest_notebook_id = dest_notebooks_dict[notebook_name]["id"]
            initial_notebook_definition = get_notebook_definition(dest_workspace_id, dest_notebook_id)
            dest_notebook_json = get_notebook_json_from_definition(initial_notebook_definition)

            # Replace parameters in the config notebook
            if notebook_name == f"{solution_name}_msft_config_notebook":
                    for index, cell in enumerate(dest_notebook_json["cells"]):
                        if "cell_type" in cell and str(cell["cell_type"]).lower() == "code" and "metadata" in cell and "tags" in cell["metadata"] and "parameters" in cell["metadata"]["tags"]:
                            source_lines = []
                            for line in cell["source"]:
                                if "=" in line:
                                    param = line.split("=")[0].replace(" ", "")
                                    if param == "workspace_name":
                                        print("updating workspace name")
                                        source_lines.append(f"workspace_name = \"{dest_workspace_id}\" \n")
                                    elif param == "solution_name":
                                        solution_id = dest_solution["id"]
                                        print("updating solution name")
                                        source_lines.append(f"solution_name = \"{solution_id}\" \n")
                                    elif param == "administration_database_name":
                                        admin_lakehouse = None
                                        print("updating admin name")
                                        for lakehouse in dest_lakehouses.keys():
                                            if "admin" in lakehouse.lower():
                                                admin_lakehouse = dest_lakehouses[lakehouse]
                                        if admin_lakehouse:
                                            admin_lakehouse_id = admin_lakehouse["id"]
                                            source_lines.append(f"administration_database_name = \"{admin_lakehouse_id}\" \n")
                                        else:
                                            source_lines.append(f"administration_database_name = \"\" \n")
                                    else:
                                        source_lines.append(line)
                                else:
                                    source_lines.append(line)

                            dest_notebook_json["cells"][index]["source"] = source_lines

                            # Only need to update the first parameters cell
                            break

            dest_notebook_json["metadata"]["dependencies"]["environment"] = env_to_attach
            if notebook_name in source_notebooks_dict:
                print(f"Found matching notebook in source workspace: {notebook_name}")
                
                source_notebook_definition = get_notebook_definition(source_workspace_id, source_notebooks_dict[notebook_name]["id"])
                source_notebook_json = get_notebook_json_from_definition(source_notebook_definition)

                try:
                    source_attached_default_lakehouse_name = source_notebook_json["metadata"]["dependencies"]["lakehouse"]["default_lakehouse_name"]
                except Exception as ex:
                    print("attached lakehouse not found in notebook json")
                    source_attached_default_lakehouse_name = None

                if source_attached_default_lakehouse_name in dest_lakehouses:
                    print(f"Found matching lakehouse to attach to notebook: {notebook_name}")
                    new_default_id = dest_lakehouses[source_attached_default_lakehouse_name]["id"]
                
                    print("Current metadata:")
                    print(json.dumps(dest_notebook_json["metadata"]["dependencies"]["lakehouse"], indent=2))

                    dest_notebook_json["metadata"]["dependencies"]["lakehouse"] = {
                        "default_lakehouse": new_default_id,
                        "default_lakehouse_name": source_attached_default_lakehouse_name,
                        "default_lakehouse_workspace_id": dest_workspace_id
                    }

                    print("Attaching new lakehouse to notebook:")
                    print(json.dumps(dest_notebook_json["metadata"]["dependencies"]["lakehouse"], indent=2))
                else:
                    dest_notebook_json["metadata"]["dependencies"]["lakehouse"] = {
                        "default_lakehouse": "",
                        "default_lakehouse_name": "",
                        "default_lakehouse_workspace_id": dest_workspace_id
                    }
                    print("Cleaning notebook metadata:")
                    print(json.dumps(dest_notebook_json["metadata"]["dependencies"]["lakehouse"], indent=2))
                
            else:
                print("Matching notebook not found in source workspace, clearing attached lakehouses from notebook")
                dest_notebook_json["metadata"]["dependencies"]["lakehouse"] = {
                    "default_lakehouse": "",
                    "default_lakehouse_name": "",
                    "default_lakehouse_workspace_id": dest_workspace_id
                }
            
            upload_notebook(dest_workspace_id, dest_notebook_id, notebook_name, initial_notebook_definition, dest_notebook_json)
            print("\n")

def publish_environment_by_id(workspace_id: str, environment_name: str, environment_id: str) -> None:
    """Summary: A helper method to publish an environment by id.

    Args:
        workspace_id (str): The workspace id.
        environment_name (str): The environment name.
        environment_id (str): The environment id.
    """
    fabric_client = FabricRestClient()
    url = f"/v1/workspaces/{workspace_id}/environments/{environment_id}/staging/publish"
    publish_response = fabric_client.post(url)
    if publish_response.ok:
        print(f"Environment {environment_name} started publishing, this operation can several minutes to complete.")
        print(f"Please check environment publishing is complete before running any notebooks or pipelines.")
    else:
        print("Error occurred when attempting to publish environment.")
        print(publish_response.status_code)
        print(publish_response.content)

def publish_environment(destination_workspace_id: str, solution_name: str = "healthcare1") -> None:
    """A helper method to publish the destination workspace.

    Args:
        destination_workspace_id (str): The destination workspace id.
        solution_name (str, optional): The healthcare data solution name. Defaults to "healthcare1".
    """

    target_environment = None
    environments = get_workspace_environments(destination_workspace_id)

    for env in environments.keys():
        if solution_name in env:
            target_environment = environments[env]
    
    publish_environment_by_id(dest_workspace_id, target_environment["displayName"], target_environment["id"])

def copy_tables(source_lakehouse_data: Any, dest_lakehouse_data: Any) -> None:
    """Summary: A helper method to copy (create emtpty) delta tables from a source lakehouse to a destination lakehouse.

    Args:
        source_lakehouse_data (Any): The source lakehouse data.
        dest_lakehouse_data (Any): The destination lakehouse data.
    """

    lakehouse_dfs = get_lakehouse_dfs_domain(dest_lakehouse_data)

    source_lakehouse_id = source_lakehouse_data["id"]
    dest_lakehouse_id = dest_lakehouse_data["id"]

    source_tables_path = f"abfss://{source_workspace_id}@{lakehouse_dfs}/{source_lakehouse_id}/Tables"
    dest_tables_path = f"abfss://{dest_workspace_id}@{lakehouse_dfs}/{dest_lakehouse_id}/Tables"

    # Iterate over the table in the source lakehouse
    tables = mssparkutils.fs.ls(source_tables_path)
    for table in tables:
        table_name = table.path.split("/")[-1]

        source_table_path = f"{source_tables_path}/{table_name}"
        dest_table_path = f"{dest_tables_path}/{table_name}"
        
        spark.sql(f'CREATE TABLE delta.`{dest_table_path}` SHALLOW CLONE delta.`{source_table_path}`')

        # Shallow clone will contrain a reference to the source table
        # Calling delete will clear the table data but keep the schema and config
        # The delta table history will show a Clone and Delete operation.
        delta_table = DeltaTable.forPath(spark, f"abfss://{dest_workspace_id}@{lakehouse_dfs}/{dest_lakehouse_id}/Tables/{table_name}")
        delta_table.delete("true")

def create_lakehouse_tables(source_workspace_id: str, dest_workspace_id: str, solution_name = "healthcare1") -> None:
    """Summary: A helper method to create lakehouse tables in a destination workspace.

    Args:
        source_workspace_id (str): The source lakehouse id.
        dest_workspace_id (str): The destination lakehouse id.
        solution_name (str, optional): The healthcare data solution name. Defaults to "healthcare1".
    """
    source_lakehouses = get_workspace_lakehouses(source_workspace_id)
    dest_lakehouses = get_workspace_lakehouses(dest_workspace_id)

    for dest_lakehouse in dest_lakehouses.keys():
        if solution_name in dest_lakehouse and dest_lakehouse in source_lakehouses:
            print(f"Creating tables for lakehouse: {dest_lakehouse}")
            source_lakehouse_data = get_lakehouse(source_workspace_id, source_lakehouses[dest_lakehouse]["id"])
            dest_lakehouse_data = get_lakehouse(dest_workspace_id, dest_lakehouses[dest_lakehouse]["id"])

            copy_tables(source_lakehouse_data, dest_lakehouse_data)
            print("\n")


#### Copy Deployment Parameters Configuration

Run the following cell to show what the update deployment parameters configuration will look like before saving to the destination workspace.


In [ ]:
updated_config, replacements = get_updated_deployment_parameters_configuration(source_workspace_id, source_admin_lakehouse_id, dest_workspace_id, dest_admin_lakehouse_id, solution_name)

print("Here are the artifacts that were udpated with their destination ids:\n")

for replaced_artifact, replaced_artifact_id in replacements.items():
    print(f"{replaced_artifact}: {replaced_artifact_id}")

print("\n")
print("This is the updated deployment parameters configuration for the destination workspace:\n")
print(json.dumps(updated_config, indent=2))

#### Save the deployment parameters to the destination workspace

After reviewing the updated deployment parameters configuration, run the following cell to persist the updated configuration to the destination workspace. Please note this will override the existing `deploymentParametersConfiguration.json` file if it already exists in that location.

In [ ]:
from os import path
system_configuration_source_path = get_system_configurations_path(source_workspace_id, source_admin_lakehouse_id)
system_configuration_destination_path = path.dirname(get_system_configurations_path(dest_workspace_id, dest_admin_lakehouse_id))
deployment_parameters_destination_path = get_deployment_parameters_destination_path(dest_workspace_id, dest_admin_lakehouse_id)

mssparkutils.fs.cp(system_configuration_source_path, system_configuration_destination_path, recurse=True)
mssparkutils.fs.put(file = deployment_parameters_destination_path, content = json.dumps(updated_config), overwrite=True)


#### Copy System Data
Execute this cell to copy important system metadata and libraries to the destination workspace.

In [ ]:
copy_workload_system_data(source_workspace_id, dest_workspace_id, dest_admin_lakehouse_id, solution_name)

#### Copy Data Assets
Execute this cell to copy sample and reference data to the destination workspace.

In [ ]:
copy_data_assets(source_workspace_id, dest_workspace_id, solution_name)

#### Create Folders
Execute this cell to copy the folder structure of the Bronze lakehouse.

In [ ]:
create_bronze_folders(source_workspace_id, source_admin_lakehouse_id, dest_workspace_id, solution_name)

#### Publish Environment

An environment was replicated by source control, but it still needs to be officially published. Note that publishing can take a long time and this code is simply starting that process but not awaiting it's completion. You can check on the publish status by navigating to the environment artifact in the Fabric UI.

In [ ]:
publish_environment(dest_workspace_id, solution_name)

#### Update Notebooks (Optional)
Execute this cell to update notebook metadata and parameter values. This step is optional as these notebooks can be updated manually. Notebooks will update their attached resources (lakehouses and environments).

In [ ]:
update_notebooks(source_workspace_id, dest_workspace_id)

#### Setup Delta Tables (Optional)
Execute this cell to create new delta tables in the destination workspace with the same schema and properties as those in the source workspace.

**NOTE**: This does not copy the data, the tables will have a matching schema and properties but will be empty.

In [ ]:
create_lakehouse_tables(source_workspace_id, dest_workspace_id)